In [1]:
import torch
from torch import nn
from torch.nn import functional as F

[**nn.Sequential和nn.Linear都是nn.Module的子类**]  
nn.Sequential定义了一种特殊的Module， 即在PyTorch中表示一个块的类， 它维护了一个由Module组成的有序列表。

In [2]:
net = nn.Sequential(nn.Linear(20,256),nn.ReLU(), nn.Linear(256,10))
X=torch.rand(2,20)
print(X)
# 当你像函数一样调用这个层的实例时，Pytorch会自动调用forward函数
# 这是因为nn.Module类重载了__call__函数
net(X)

tensor([[0.0732, 0.1416, 0.6206, 0.2854, 0.2596, 0.6025, 0.2190, 0.7770, 0.6382,
         0.6264, 0.6551, 0.1717, 0.5964, 0.1889, 0.3372, 0.2004, 0.4985, 0.3251,
         0.7591, 0.1905],
        [0.0231, 0.6089, 0.4982, 0.1456, 0.0419, 0.2254, 0.5718, 0.8016, 0.7208,
         0.4983, 0.4431, 0.8389, 0.7194, 0.9320, 0.6088, 0.1000, 0.4035, 0.0782,
         0.2163, 0.1977]])


tensor([[-1.6984e-02,  9.1910e-05, -5.3518e-02, -1.7719e-01, -1.5649e-01,
          4.3066e-02, -1.4175e-01, -2.5307e-01,  2.3861e-02, -1.2522e-01],
        [-1.4696e-02, -7.3726e-02, -8.0044e-02, -1.6672e-01, -1.2479e-01,
          1.4736e-02, -2.3628e-01, -3.0702e-01,  1.5554e-01, -1.8936e-01]],
       grad_fn=<AddmmBackward0>)

In [5]:
# 继承自Module类的方式
class MLP(nn.Module):
    def __init__(self):
        # 调用MLP父类Module的构造函数来进行必要的初始化
        super().__init__()
        # 声明两个全连接层
        self.hidden = nn.Linear(20,256)
        self.out = nn.Linear(256,10)
    
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))

In [6]:
net = MLP()
net(X)

tensor([[ 0.0437,  0.0544,  0.1791, -0.0939, -0.0137, -0.2055, -0.0396,  0.1515,
         -0.0580,  0.1183],
        [ 0.0301, -0.1073,  0.1183, -0.0726, -0.0579, -0.1767,  0.0094,  0.2949,
         -0.0366,  0.0680]], grad_fn=<AddmmBackward0>)

[**手动实现Sequential**]  
回想一下Sequential的设计是为了把其他模块（module）串起来。  
为了构建我们自己的简化的MySequential， 我们只需要定义两个关键函数：  
1.一种将块逐个追加到列表中的函数；  
2.一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

In [7]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 向pytorch模型module中动态添加模块，模块本身也是一个Module类的实例
            # nn.Module类中的_modules是一个OrderedDict(有序字典)，可以保证添加进去的模块的顺序是固定的
            # 这里idx是一个int类型，不能直接作为key，需要转换成str类型
            self._modules[str(idx)]=module
    # 实现前向传播
    def forward(self,X):
        # OrderedDict保证了按照添加进去的顺序遍历
        for block in self._modules.values():
            # 链式调用每个模块的forward方法
            X = block(X)
        return X

In [8]:
net = MySequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256,10))
net(X)

tensor([[ 0.1023,  0.0211, -0.0037,  0.0179, -0.3167, -0.0294,  0.0846, -0.0887,
          0.0402,  0.0313],
        [ 0.0983, -0.0075, -0.0829, -0.0069, -0.4463, -0.1496,  0.0671, -0.0290,
         -0.0013,  0.1063]], grad_fn=<AddmmBackward0>)

In [9]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20),requires_grad = False)
        # 在输出之前添加一个全连接层
        self.linear = nn.Linear(20,20)
    
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常数参数以及relu函数
        X = F.relu(torch.mm(X,self.rand_weight)+1)
        # 复用全连接层，这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [10]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0978, grad_fn=<SumBackward0>)

In [13]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64), nn.ReLU(),
                                 nn.Linear(64,32), nn.ReLU())
        self.linear = nn.Linear(32,16)
        
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.3464, grad_fn=<SumBackward0>)